## Step 7: 역전파 자동화

앞서 우리는 역전파를 직접 구현해보았다. 하지만 기존 방식은 매 계산마다 역전파 계산 코드를 직접 작성해줘야 했기 때문에 불편하고 실수가 생길 확률도 많았다. 따라서 이번에는 순전파만 계산하면 어떤 계산에 대해서든 역전파가 자동으로 될 수 있도록 해보겠다.

> **Define-by-Run**
>
> 딥러닝에서 수행하는 계산들을 계산 시점에 연결하는 방식으로, '동적 계산 그래프'라고도 한다.

### 7.1 역전파 자동화의 시작

역전파 자동화로 가는 길은 변수와 함수의 관계를 이해하는 데서 출발한다. 

함수 입장에서 변수는 '입력'과 '출력'에 사용된다. 즉, 함수에게 변수는 '입력 변수(input)'와 '출력 변수(output)'으로서 존재한다.

또 변수 관점에서는, 함수는 자신을 만든 창조자(creator) 혹은 부모 같은 존재이다. 기본적으로 변수는 창조자 함수가 존재하며, 창조자가 존재하지 않는 변수는 함수 외 존재, 즉 사용자에 의해서 만들어진 변수로 간주된다.

이러한 관계를 계산 시점에 맺어주면 역전파 자동화 구현을 할 수 있다.

```python
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
```

creator라는 인스턴스 변수를 추가하였고, creator를 설정할 수 있도록 set_creator 메서드도 추가하였다. 이어서 Function 클래스에 이를 추가해주도록 하겠다.

```python
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 출력 변수에 창조자 설정
        self.input = input
        self.output = output # 출력 저장
        return output
```

순전파를 계산하면 결과를 output이라는 Variable 인스턴스가 생성된다. 이때 이 output에 `set_creator`를 통해 관계를 기억시킨다. 그리고 다음 계산을 위해 output을 저장한다.

이와 같이 '연결'된 Variable과 Function이 있다면 계산 그래프를 역순으로 진행할 수 있다.

```python
A = Square()
B = Exp()
C = Square()

# 순전파
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 역순으로 올라감
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x
```

위 코드는 변수에서 creator를 거쳐 함수로, 함수에서 input을 거쳐 변수로 거슬러 올라가는 코드이다. 이처럼 실제 계산을 수행하는 시점에 연결이 만들어지는 것을 Define-by-Run이라고 한다. 계산이 수행(Run)됨에 따라 연결이 정의(Define)되는 것이다.


### 7.2 역전파 도전!

앞서 설명한 변수와 함수의 관계를 이용하여 역전파를 해보도록 하자.

```python
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져온다
b = C.input # 2. 함수의 입력, 즉 입력 변수를 가져온다
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다

B = b.creator # 1. 함수를 가져온다
a = B.input # 2. 함수의 입력, 즉 입력 변수를 가져온다
a.grad = B.backward(b.grad) # 3. 함수의 backward 메서드를 호출한다

A = a.creator
x = A.input
x.grad = A.backward(a.grad)
```

y의 인스턴스 변수 creator에서 함수를 얻어오고, 그 함수의 input에서 입력 변수를 가져왔다. 이후 backward 메서드를 호출하였다. 같은 과정을 b에서 a로, a에서 x로의 과정에도 적용하면 된다.

```python
print(x.grad)
```
\> 3.297442541400256

### 7.3 backward 메서드 추가

앞서 진행했던 코드를 살펴보면, 반복되는 패턴이 눈에 띈다. 이 패턴을 자연스럽게 반복할 수 있도록 한다면 자동화 구현이 가능할 것 같다.

```python
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator # 1
        if f is not None: # f가 역순 방향 마지막 연산이 아니면
            x = f.input # 2
            x.grad = f.backward(self.grad) # 3
            x.backward() # 하나 앞 변수의 backward 메서드를 재귀적으로 호출한다
```

앞서 우리가 진행했던 내용을 그대로 구현하였다. Variable의 creator에서 함수를 얻어오고, 그 함수의 입력 변수를 가져온다. 이후 함수의 backward 메서드를 호출하고 자신보다 하나 앞 변수의 backward 메서드를 호출한다. 이 과정을 마지막 함수(creator가 None)에 다다르기까지 재귀적으로 반복하면 자동 역전파가 구현된다.

```python
A = Square()
B = Exp()
C = Square()

# 순전파
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)
```
\> 3.297442541400256